---
title: Neural Estimation via DV bound
math:
  '\abs': '\left\lvert #1 \right\rvert'
  '\norm': '\left\lvert #1 \right\rvert'
  '\Set': '\left\{ #1 \right\}'
  '\mc': '\mathcal{#1}'
  '\M': '\boldsymbol{#1}'
  '\R': '\mathsf{#1}'
  '\RM': '\boldsymbol{\mathsf{#1}}'
  '\op': '\operatorname{#1}'
  '\E': '\op{E}'
  '\d': '\mathrm{\mathstrut d}'
---

Since the $f$-divergence is nothing but an expectation, it may be tempting to approximate it by the sample average

$$
\begin{align}
\lim_{n\to \infty}\frac1n \sum_{i\in [n]} f\left(\frac{d P_{\R{Z}}}{d P_{\R{Z}'}}(\R{Z}'_i)\right) \xlongequal{\text{a.s.}}
E\left[f\left(\frac{dP_{\R{Z}}}{dP_{\R{Z}'}}(\R{Z}')\right)\right] = D_f(P_{\R{Z}} \| P_{\R{Z}'}), 
\end{align}
$$ (eq:avg-f-D)

where the almost sure convergence to [](#eq:f-D) is by the law of large number in [](#thm:SLLN). 

::::{caution}

Despite having the desired convergence, the sample average in [](#eq:avg-f-D) is invalid because calculating the sample average requires the knowledge of the unknown density ratio of $P_{\R{Z}}$ to $P_{\R{Z}'}$.

::::

This chapter aims to carefully unroll the idea of the neural estimation of divergence, namely, to train a neural network to tighten certain bound on the divergence.

## Neural estimation of KL divergence

The following result rewrites the KL divergence as an optimization over probability measures.

::::{prf:proposition}
:label: pro:DV1

The KL divergence in [](#eq:D) can be written as

$$
\begin{align}
D(P_{\R{Z}}\|P_{\R{Z}'}) & =  \sup_{Q\sim P_{\R{Z}}} E \left[ \log \frac{dQ}{dP_{\R{Z}'}}(\R{Z}) \right],
\end{align}
$$ (eq:D1)

which has a unique solution $Q=P_{\R{Z}}$.

::::

::::{prf:proof}
:class: dropdown
:nonumber: true

To prove [](#eq:D1),

$$
\begin{align*}
D(P_{\R{Z}}\|P_{\R{Z}'})  &= D(P_{\R{Z}}\|P_{\R{Z}'}) - \inf_{Q\sim P_{\R{Z}}} \underbrace{D(P_{\R{Z}}\|Q)}_{\geq 0 \text{ with equality iff } Q=P_{\R{Z}}\kern-3em} \\
&= \sup_{Q\sim P_{\R{Z}}}  \underbrace{D(P_{\R{Z}}\|P_{\R{Z}'})}_{=E \left[\log\frac{dP_{\R{Z}}}{dP_{\R{Z}'}}(\R{Z})\right]} -  \underbrace{D(P_{\R{Z}}\|Q)}_{=E \left[\log\frac{dP_{\R{Z}}}{dQ}(\R{Z})\right]}\\
&= \sup_{Q\sim P_{\R{Z}}} E \left[\log\frac{dQ}{dP_{\R{Z}'}}(\R{Z})\right]
\end{align*}
$$

where the first inequality is by the positivity of KL divergence. The equality condition gives the unique solution as desired.

::::

::::{note}

[](#eq:D1) is the same as the definition in [](#eq:D) except that $P_{\R{Z}}$ is replaced by an arbitrary parameter $Q$.

- It essentially gives a tight lower bound on KL divergence.
- The unknown distribution is recovered as the optimal solution.

::::

The idea of neural estimation is to 

1. estimate the expectation in [](#eq:D1) by the sample average
  
  $$
  \frac1n \sum_{i\in [n]} \log \underbrace{\frac{dQ}{dP_{\R{Z}'}}(\R{Z})}_{\text{(*)}},
  $$
  
2. use a neural network to compute the density ratio (*), and
3. train the network to maximize the expectation estimate, e.g., by gradient ascent on the above sample average.

## Donsker-Varadhan formula

**How to compute (*) without the knowledge of $P_{\R{Z}'}$?**

Consider a change of variable such as

$$
r(z) = \frac{dQ(z)}{dP_{\R{Z}'}(z)}
$$ (eq:Q-r)

to absorb the unknown reference into the parameter, which gives the following expression for KL divergence.

::::{prf:proposition}
:label: pro:DV2

$$
\begin{align}
D(P_{\R{Z}}\|P_{\R{Z}'}) & =  \sup_{\substack{r\in \mathbb{R}^{\Omega_{\R{Z}}}_+\\ E[r(\R{Z}')]=1}} E \left[ \log r(\R{Z}) \right] 
\end{align}
$$ (eq:D2)

where the solution $r$ satisfies 
$
r(\R{Z}) \xlongequal{\text{a.s.}} \frac{dP_{\R{Z}}}{dP_{\R{Z}'}}(\R{Z}).
$ 

::::

::::{prf:proof}
:class: dropdown
:nonumber: true

To prove [](#eq:D2), substitute [](#eq:Q-r) to [](#eq:D1). The constraint on $r$ is obtained from the constraint on $\frac{dQ}{dP_{\R{Z}}}$ as follows:

$$
\begin{align*}
\frac{dQ}{dP_{\R{Z}}}(z) \geq 0 &\iff r(z)\geq 0\\
\int_{\Omega_{\R{Z}}}\frac{dQ}{dP_{\R{Z}}}dP_{\R{Z}}=1 &\iff E[r(\R{Z}')]=1.
\end{align*}
$$



::::

The next step is to have a neural network compute $r$ and train it to maximize the sample average estimate

$$
\begin{align}
\sup_{\substack{r\in \mathbb{R}^{\Omega_{\R{Z}}}_+\\ \frac1{n'}\sum_{i\in [n']} r(\R{Z}'_i)]=1}} \frac1n \sum_{i\in [n]} \log r(\R{Z}_i).
\end{align}
$$ (eq:avg-D1)

**How to impose the constraint on $r$ when training a neural network?**

Consider another change of variable

$$
\begin{align}
r(z)&=\frac{e^{t(z)}}{E[e^{t(\R{Z}')}]}.
\end{align}
$$ (eq:r-t)

::::{exercise}
:label: ex:r-t

Show that $r$ defined in [](#eq:r-t) satisfies the constraint in [](#eq:D2) for all real-valued function $t\in \mathbb{R}^{\Omega_{\R{Z}}}$.

::::

YOUR ANSWER HERE

Substituting [](#eq:r-t) into [](#eq:D2) gives the well-known *Donsker-Varadhan (DV)* formula [](https://doi.org/10.1002/cpa.3160360204):

::::{prf:theorem} DV formula
:label: thm:DV

$$
\begin{align}
D(P_{\R{Z}}\|P_{\R{Z}'}) =  \sup_{t\in \mathbb{R}^{\Omega_{\R{Z}}}} E[t(\R{Z})] - \log E[e^{t(\R{Z}')}]
\end{align}
$$ (eq:DV)

where the optimal $t$ satisfies

$$
\begin{align}
t(\R{Z}) \xlongequal{\text{a.s.}} \log \frac{dP_{\R{Z}}}{dP_{\R{Z}'}}(\R{Z}) + c
\end{align}
$$ (eq:DV:sol)

almost surely for some constant $c$.

::::

::::{prf:proof}
:class: dropdown
:nonumber: true

To prove [](#eq:DV), substitute [](#eq:r-t) to [](#eq:D2) and argue that the constraint of $r$ in [](#eq:D2) is satisfied by [](#eq:r-t) as in [](#ex:r-t).

::::

The divergence can be estimated as follows instead of [](#eq:avg-D1):

$$
\begin{align}
\sup_{t\in \mathbb{R}^{\Omega_{\R{Z}}}} \frac1n \sum_{i\in [n]} t(\R{Z}_i) - \frac1{n'}\sum_{i\in [n']} e^{t(\R{Z}'_i)}
\end{align}
$$ (eq:avg-DV)

In summary, the neural estimation of KL divergence is a sample average of [](#eq:D) but with the unknown density ratio replaced by [](#eq:r-t)

$$
D(P_{\R{Z}}\| P_{\R{Z}'}) = \underset{\stackrel{\uparrow}\sup_t}{} \overbrace{E}^{\op{avg}} \bigg[ \log \underbrace{\frac{P_{\R{Z}}}{P_{\R{Z}'}}(\R{Z})}_{\frac{e^{t(\R{Z})}}{\underbrace{E}_{\op{avg}}[e^{t(\R{Z}')}]}} \bigg].
$$

but with the unknown density ratio replaced by [](#eq:r-t) trained as a neural network.